# About
* Date created: 23 Nov 2021
* Ref: https://towardsdatascience.com/python-powered-monte-carlo-simulations-fc3c71b5b83f

Objective
* be able to implement simple MC simulation using scipy

# Import

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from scipy.stats import qmc    # quasi-Monte Carlo for latin hypercube sampling
from scipy.stats import mstats 
from scipy import stats as stats
from scipy.stats import rv_continuous
import scipy.optimize as opt

from scipy.stats import norm, weibull_min, beta


import warnings
warnings.filterwarnings("ignore")

In [2]:
ksN = 100

# 1. Theory Primer

## What are Monte-carlo simulations?
* The Monte-Carlo(MC) method is a simulation technique that constructs probability distributions for the output variables of a model in which some of the input arguments are random variables
* MC simulation is based on repeated random sampling and the underlying concept of MC is to use <b> randomness </b> to solve problems that may be deterministic in principle
* It is a popular techniques to draw inferences about a population without knowing the true underlying population distribution. 
    * * The MC method is sometimes called a multiple probability simulation technique because it integrates multiple random variables whose combined effects cannot easily be described by a closed-form equation.
        * The MC method was invented in the late 1940s by John von Neumann and Stanislaw Ułam while they were working at the Los Alamos Laboratory. They hit a dead-end when they tried to compute neutron collisions by applying deterministic methods.
* Monte Carlo has 3 main usages: 
    * estimate parameters or statistical measures, 
    * examine the properties of the estimates, 
    * approximate integrals
    
* One of the underlying ideas behind MC is that: as we use more samples, our answers should increasingly become more accurate
    * Intuitively this makes sense. When flipping a coin for Heads or Tails, with a larger number of total flips we are more likely to approach the true distribution

### Example of MC simulation problem
*  An enterprise means to simulate the future success of a new product and builds the simulation model as follows:
    * the expected sales volume, v, as a random variable that follows a beta-PERT distribution, derived from a 3-point-estimate;
    * the price p of the product will be subject to negotiations — some customers will be in a position to claim volume discounts or early payment discounts; the planners decide to model the uncertainty about the average price the company will realize as a normal random variable around a mean of 20 and a standard deviation of 2;
    * The raw material costs, m, of the product will be subject to upcoming supplier price increases, to be estimated by another normal distribution;
    * The model should provide multiple outcomes: revenues, profit, total cost.

# 2. Monte Carlo Estimation
* Monte Carlo uses the Law of Large Numbers (LLN) to come up with an estimate for a population parameter using simulated values. 
* Law of Large Numbers
    * Suppose we have a set {X1,X2,X3....,Xn}, and they are all independent random variables with the same underlying distribution -> independent, identically distributed (i.i.d.), where all X's have the same mean and standard deviation
    * As the sample size grows, the probability that the average of all X’s is equal to the mean μ is equal to 1.
* Consequently, the idea behind Monte Carlo estimation is that when we obtain an estimate for a parameter a large number of times, let’s say M = 10000 times, then the mean of these estimates will form a Monte Carlo unbiased estimate for that parameter.

# Monte-Carlo Estimation: Python Implementation
* Let’s say we want to estimate a causal effect between two variables, pair of independent and dependent variables and we have some idea about possible values of intercept alpha and slope parameter beta.
* We can randomly sample from normal distribution to generate error terms, dependent and independent variable values. * Then we can estimate the coefficient of beta, beta_hat, and repeat this process until M = 10000 times.
* By the LLN, the sample mean of these 10000 beta_hats will be an unbiased estimate for the true beta. That is:

$$
    Y = \alpha + \beta X + \epsilon\\
    \hat{\beta}_{MC} = \frac{1}{N} \sum_{i=1}^{N}\ \hat{\beta}
$$

In [11]:
import statsmodels.api as sm

np.random.seed(2021)

mu = 0
sigma = 1
n = 100

# Assume some population parameters
alpha = np.repeat(0.5,n)
beta = 1.5

def MC_ESTIMATION_SLOPE(M):
    MC_BETAS = []
    MC_SAMPLES = {}
    
    for i in range(M):
        #random sampling from normal distribution to get error terms
        e = np.random.normal(mu,sigma,n)
        
        #generate independent variable by making sure variance_X is larger than variance_error
        X = 9* np.random.normal(mu,sigma,n)
        
        #population distribution using the assumed parameter values alpha and beta
        Y = (alpha + beta * X + e)
        
        # run OLS regression to get slope parameters
        model = sm.OLS(Y.reshape((-1, 1)), X.reshape((-1, 1)))
        ols_result = model.fit()
        coeff = ols_result.params
        
        MC_SAMPLES[i] = Y
        MC_BETAS.append(coeff)
        
    MC_BETA_HATS = np.array(MC_BETAS).flatten()
    return(MC_SAMPLES,MC_BETA_HATS)

In [12]:
MC_SAMPLES, MC_BETA_HATS = MC_ESTIMATION_SLOPE(M = 10000)
beta_hat_MC = np.mean(MC_BETA_HATS)

In [14]:
MC_BETA_HATS

array([1.49450864, 1.47494779, 1.50035304, ..., 1.50746915, 1.49457376,
       1.49608805])

In [15]:
beta_hat_MC

1.5002180550943296